In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.neural_network import MLPClassifier
import warnings
warnings.filterwarnings('ignore')

In [11]:
confirmed_planets = pd.read_csv("Confirmed_planets.csv")
confirmed_planets_2 = pd.read_csv("Confirmed_planets.csv")
candidate_planets = pd.read_csv("Candidate_planets.csv")
candidate_planets_2 = pd.read_csv("Candidate_planets.csv")

# Preprocessing Confirmed planets

In [12]:
confirmed_planets.head()

,name,planet_status,mass,mass_error_min,mass_error_max,mass_sini,mass_sini_error_min,mass_sini_error_max,radius,radius_error_min,...,star_sp_type,star_age,star_age_error_min,star_age_error_max,star_teff,star_teff_error_min,star_teff_error_max,star_detected_disc,star_magnetic_field,star_alternate_names
0,11 Com b,Confirmed,16.1284,1.53491,1.53491,16.1284,1.53491,1.53491,NaN,NaN,...,G8 III,NaN,NaN,NaN,4742.0,100.0,100.0,NaN,NaN,NaN
1,11 Oph b,Confirmed,21.0000,3.00000,3.00000,NaN,NaN,NaN,NaN,NaN,...,M9,0.011,0.002,0.002,2375.0,175.0,175.0,NaN,NaN,"Oph 1622-2405, Oph 11A"
2,11 UMi b,Confirmed,11.0873,1.10000,1.10000,11.0873,1.10000,1.10000,NaN,NaN,...,K4III,1.560,0.540,0.540,4340.0,70.0,70.0,NaN,NaN,NaN
3,14 And b,Confirmed,4.6840,0.23000,0.23000,4.6840,0.23000,0.23000,NaN,NaN,...,K0III,NaN,NaN,NaN,4813.0,20.0,20.0,NaN,NaN,NaN
4,14 Her b,Confirmed,NaN,NaN,NaN,NaN,4.95000,4.95000,NaN,NaN,...,K0 V,5.100,NaN,NaN,5311.0,87.0,87.0,NaN,NaN,NaN


In [13]:
print(confirmed_planets.shape)
for each_column in confirmed_planets.columns:
#     print(f"{each_column} : {confirmed_planets[each_column].isna().sum()}/{len(confirmed_planets[each_column])}")
    if confirmed_planets[each_column].isna().sum() > 2700:
        confirmed_planets = confirmed_planets.drop([each_column], axis = 1)
        
for each_column in confirmed_planets.columns:        
    print(f"{each_column} : {confirmed_planets[each_column].isna().sum()}/{len(confirmed_planets[each_column])}")
print(confirmed_planets.shape)


(5638, 98)
name : 0/5638
planet_status : 0/5638
mass : 2547/5638
radius : 1613/5638
radius_error_min : 1776/5638
radius_error_max : 1776/5638
orbital_period : 514/5638
orbital_period_error_min : 779/5638
orbital_period_error_max : 779/5638
semi_major_axis : 1815/5638
discovered : 12/5638
updated : 0/5638
publication : 0/5638
detection_type : 0/5638
alternate_names : 2255/5638
star_name : 116/5638
ra : 0/5638
dec : 0/5638
mag_j : 2700/5638
star_distance : 362/5638
star_distance_error_min : 1991/5638
star_distance_error_max : 1991/5638
star_metallicity : 1192/5638
star_metallicity_error_min : 2098/5638
star_metallicity_error_max : 2098/5638
star_mass : 637/5638
star_mass_error_min : 1444/5638
star_mass_error_max : 1444/5638
star_radius : 1004/5638
star_radius_error_min : 1204/5638
star_radius_error_max : 1204/5638
star_age : 2651/5638
star_teff : 805/5638
star_teff_error_min : 1031/5638
star_teff_error_max : 1031/5638
star_alternate_names : 2248/5638
(5638, 36)


In [14]:

confirmed_planets.head()


,name,planet_status,mass,radius,radius_error_min,radius_error_max,orbital_period,orbital_period_error_min,orbital_period_error_max,semi_major_axis,...,star_mass_error_min,star_mass_error_max,star_radius,star_radius_error_min,star_radius_error_max,star_age,star_teff,star_teff_error_min,star_teff_error_max,star_alternate_names
0,11 Com b,Confirmed,16.1284,NaN,NaN,NaN,326.03,0.32,0.32,1.29,...,0.300,0.300,19.000,2.000,2.000,NaN,4742.0,100.0,100.0,NaN
1,11 Oph b,Confirmed,21.0000,NaN,NaN,NaN,730000.00,365000.00,365000.00,243.00,...,0.005,0.005,NaN,NaN,NaN,0.011,2375.0,175.0,175.0,"Oph 1622-2405, Oph 11A"
2,11 UMi b,Confirmed,11.0873,NaN,NaN,NaN,516.22,3.25,3.25,1.54,...,0.250,0.250,24.080,1.840,1.840,1.560,4340.0,70.0,70.0,NaN
3,14 And b,Confirmed,4.6840,NaN,NaN,NaN,185.84,0.23,0.23,0.83,...,0.200,0.200,11.000,1.000,1.000,NaN,4813.0,20.0,20.0,NaN
4,14 Her b,Confirmed,NaN,NaN,NaN,NaN,1767.56,0.22,0.22,2.82,...,NaN,NaN,0.708,0.085,0.085,5.100,5311.0,87.0,87.0,NaN


In [15]:
print(len(confirmed_planets_2['name'].unique()))
print(len(confirmed_planets_2))

5638
5638


In [16]:
# Removing irrelevant columns

confirmed_planets = confirmed_planets.drop(['planet_status', 'star_alternate_names','name', 'star_name','alternate_names','publication','updated'], axis = 1)

# Removing discover type and year for now

confirmed_planets = confirmed_planets.drop(['detection_type', 'discovered'], axis = 1)


In [17]:
# non_null_confirmed.shape

# Imputing low number of Nan values with median imputation first

In [18]:
print(confirmed_planets.columns)

Index(['mass', 'radius', 'radius_error_min', 'radius_error_max',
       'orbital_period', 'orbital_period_error_min',
       'orbital_period_error_max', 'semi_major_axis', 'ra', 'dec', 'mag_j',
       'star_distance', 'star_distance_error_min', 'star_distance_error_max',
       'star_metallicity', 'star_metallicity_error_min',
       'star_metallicity_error_max', 'star_mass', 'star_mass_error_min',
       'star_mass_error_max', 'star_radius', 'star_radius_error_min',
       'star_radius_error_max', 'star_age', 'star_teff', 'star_teff_error_min',
       'star_teff_error_max'],
      dtype='object')


In [20]:
median_imp_cols = ['star_teff', 'star_teff_error_min', 'star_teff_error_max',
                   'star_radius', 'star_mass', 'star_distance', 'orbital_period',
                  'orbital_period_error_min', 'orbital_period_error_max']

for each_col in median_imp_cols:
    confirmed_planets[each_col] = confirmed_planets[each_col].fillna(confirmed_planets[each_col].median())

In [21]:
for each_column in confirmed_planets.columns:        
    print(f"{each_column} : {confirmed_planets[each_column].isna().sum()}/{len(confirmed_planets[each_column])}")
print(confirmed_planets.shape)

mass : 2547/5638
radius : 1613/5638
radius_error_min : 1776/5638
radius_error_max : 1776/5638
orbital_period : 0/5638
orbital_period_error_min : 0/5638
orbital_period_error_max : 0/5638
semi_major_axis : 1815/5638
ra : 0/5638
dec : 0/5638
mag_j : 2700/5638
star_distance : 0/5638
star_distance_error_min : 1991/5638
star_distance_error_max : 1991/5638
star_metallicity : 1192/5638
star_metallicity_error_min : 2098/5638
star_metallicity_error_max : 2098/5638
star_mass : 0/5638
star_mass_error_min : 1444/5638
star_mass_error_max : 1444/5638
star_radius : 0/5638
star_radius_error_min : 1204/5638
star_radius_error_max : 1204/5638
star_age : 2651/5638
star_teff : 0/5638
star_teff_error_min : 0/5638
star_teff_error_max : 0/5638
(5638, 27)


In [22]:
# 11 cols of remainign 27 are now without nan
# Remaining 16 will be filled using regression imputation

In [23]:
# Removed all rows in nan values in em

non_nan_cols = ['star_teff', 'star_teff_error_min', 'star_teff_error_max',
                   'star_radius', 'star_mass', 'star_distance', 'orbital_period',
                  'orbital_period_error_min', 'orbital_period_error_max', 'ra', 'dec']
nan_cols = list(set(confirmed_planets.columns.to_list()) - set(non_nan_cols))
non_null_confirmed = confirmed_planets.dropna()

In [24]:
nan_cols

['star_distance_error_min',
 'star_mass_error_min',
 'radius_error_max',
 'mass',
 'star_metallicity',
 'star_distance_error_max',
 'radius_error_min',
 'star_metallicity_error_min',
 'star_radius_error_max',
 'star_mass_error_max',
 'star_radius_error_min',
 'mag_j',
 'radius',
 'semi_major_axis',
 'star_age',
 'star_metallicity_error_max']

In [25]:
#Keeping a dataset consisting all rows that have nan values

null_confirmed = confirmed_planets[confirmed_planets.isnull().any(axis=1)]


In [26]:
print(null_confirmed.shape)

(5437, 27)


## Imputing high number of Nan values using regression

In [27]:
target_var = []
for each_col in confirmed_planets:
    print(f"{each_col} : {confirmed_planets[each_col].isna().sum()}")
    if confirmed_planets[each_col].isna().sum() >= 2000:
        target_var.append(each_col)  

mass : 2547
radius : 1613
radius_error_min : 1776
radius_error_max : 1776
orbital_period : 0
orbital_period_error_min : 0
orbital_period_error_max : 0
semi_major_axis : 1815
ra : 0
dec : 0
mag_j : 2700
star_distance : 0
star_distance_error_min : 1991
star_distance_error_max : 1991
star_metallicity : 1192
star_metallicity_error_min : 2098
star_metallicity_error_max : 2098
star_mass : 0
star_mass_error_min : 1444
star_mass_error_max : 1444
star_radius : 0
star_radius_error_min : 1204
star_radius_error_max : 1204
star_age : 2651
star_teff : 0
star_teff_error_min : 0
star_teff_error_max : 0


In [28]:
target_var = nan_cols

In [29]:
non_null_confirmed.columns

Index(['mass', 'radius', 'radius_error_min', 'radius_error_max',
       'orbital_period', 'orbital_period_error_min',
       'orbital_period_error_max', 'semi_major_axis', 'ra', 'dec', 'mag_j',
       'star_distance', 'star_distance_error_min', 'star_distance_error_max',
       'star_metallicity', 'star_metallicity_error_min',
       'star_metallicity_error_max', 'star_mass', 'star_mass_error_min',
       'star_mass_error_max', 'star_radius', 'star_radius_error_min',
       'star_radius_error_max', 'star_age', 'star_teff', 'star_teff_error_min',
       'star_teff_error_max'],
      dtype='object')

In [30]:
for each_col in non_null_confirmed.columns:
    print(non_null_confirmed[each_col][:5])

144    0.070
259    0.840
262    0.085
285    0.013
487    0.591
Name: mass, dtype: float64
144    0.230
259    1.115
262    0.514
285    0.170
487    0.827
Name: radius, dtype: float64
144    0.0100
259    0.0570
262    0.0270
285    0.0058
487    0.0550
Name: radius_error_min, dtype: float64
144    0.0100
259    0.0570
262    0.0270
285    0.0058
487    0.0550
Name: radius_error_max, dtype: float64
144    41.685500
259    20.273034
262    19.492210
285     5.745998
487     2.899703
Name: orbital_period, dtype: float64
144    0.003100
259    0.000036
262    0.000653
285    0.000002
487    0.000054
Name: orbital_period_error_min, dtype: float64
144    0.003100
259    0.000036
262    0.000653
285    0.000002
487    0.000054
Name: orbital_period_error_max, dtype: float64
144    0.24100
259    0.13670
262    0.14930
285    0.04180
487    0.03866
Name: semi_major_axis, dtype: float64
144     53.650967
259    126.553444
262    136.573975
285    134.718109
487    206.095833
Name: ra, dtype: 

In [31]:
non_null_confirmed.shape

(201, 27)

In [34]:
x_train = non_null_confirmed.drop(target_var, axis = 1)
x_train.columns = x_train.columns.astype(str)
y_train = non_null_confirmed[target_var]

In [35]:
y_train.head()

,star_distance_error_min,star_mass_error_min,radius_error_max,mass,star_metallicity,star_distance_error_max,radius_error_min,star_metallicity_error_min,star_radius_error_max,star_mass_error_max,star_radius_error_min,mag_j,radius,semi_major_axis,star_age,star_metallicity_error_max
144,1.25,0.40,0.0100,0.070,-0.15,1.25,0.0100,0.05,0.060,0.40,0.060,9.770,0.230,0.24100,3.34,0.05
259,4.20,0.02,0.0570,0.840,-0.10,4.20,0.0570,0.05,0.010,0.02,0.010,12.108,1.115,0.13670,10.70,0.05
262,1.29,0.04,0.0270,0.085,0.10,1.29,0.0270,0.10,0.020,0.04,0.020,9.144,0.514,0.14930,3.99,0.10
285,0.04,0.02,0.0058,0.013,0.04,0.04,0.0058,0.11,0.010,0.02,0.010,10.058,0.170,0.04180,9.90,0.11
487,10.00,0.03,0.0550,0.591,0.41,10.00,0.0550,0.08,0.039,0.03,0.039,9.936,0.827,0.03866,1.60,0.08


In [37]:


model = LinearRegression()
model.fit(x_train, y_train)

X_pred = null_confirmed.drop(target_var, axis =1)
y_pred = model.predict(X_pred)
y_pred = pd.DataFrame(y_pred, columns = y_train.columns).applymap(lambda x: abs(x) if x < 0 else x)


In [38]:
y_pred = y_pred.set_index(X_pred.index)
imputed_df = pd.concat([X_pred,y_pred], axis = 1)
final_confirmed_exo = pd.concat([non_null_confirmed,imputed_df], axis = 0).sort_index()

final_confirmed_exo.shape


(5638, 27)

In [39]:
print(non_null_confirmed.shape)


(201, 27)


In [40]:
print(y_pred.shape)
print(X_pred.shape)

(5437, 16)
(5437, 11)


# Non Exo planets

In [43]:
non_exo = pd.read_csv("Confirmed_non_exoplanets.csv")

In [44]:
non_exo.head()

,name,planet_status,mass,mass_error_min,mass_error_max,mass_sini,mass_sini_error_min,mass_sini_error_max,radius,radius_error_min,...,star_sp_type,star_age,star_age_error_min,star_age_error_max,star_teff,star_teff_error_min,star_teff_error_max,star_detected_disc,star_magnetic_field,star_alternate_names
0,1SWASPJ182438.34 + 302546.0,Controversial,NaN,NaN,NaN,NaN,NaN,NaN,1.8,0.14,...,NaN,NaN,NaN,NaN,7800.0,NaN,NaN,NaN,NaN,NaN
1,30 Ari B b,Retracted,148.0000,27.0000,27.0000,9.8800,0.9400,0.9400,NaN,NaN,...,F6V,0.91,0.03,0.03,6484.0,98.0,98.0,NaN,NaN,NaN
2,42 Dra b,Controversial,3.8800,0.8500,0.8500,3.8800,0.8500,0.8500,NaN,NaN,...,K1.5III,9.49,1.76,1.76,4200.0,70.0,70.0,NaN,NaN,NaN
3,AD Leo b,Controversial,0.0727,0.0176,0.0176,0.0727,0.0176,0.0176,NaN,NaN,...,M4V,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GJ 388
4,Aldebaran b,Controversial,6.4700,0.5300,0.5300,6.4700,0.5300,0.5300,NaN,NaN,...,K5III,6.60,2.40,2.40,4055.0,70.0,70.0,NaN,NaN,alf Tau


In [45]:
for each_column in non_exo.columns:        
    print(f"{each_column} : {non_exo[each_column].isna().sum()}/{len(non_exo[each_column])}")
print(non_exo.shape)

name : 0/105
planet_status : 0/105
mass : 32/105
mass_error_min : 47/105
mass_error_max : 47/105
mass_sini : 65/105
mass_sini_error_min : 71/105
mass_sini_error_max : 71/105
radius : 68/105
radius_error_min : 78/105
radius_error_max : 78/105
orbital_period : 12/105
orbital_period_error_min : 33/105
orbital_period_error_max : 33/105
semi_major_axis : 29/105
semi_major_axis_error_min : 53/105
semi_major_axis_error_max : 53/105
eccentricity : 49/105
eccentricity_error_min : 62/105
eccentricity_error_max : 62/105
inclination : 78/105
inclination_error_min : 92/105
inclination_error_max : 92/105
angular_distance : 78/105
discovered : 1/105
updated : 0/105
omega : 65/105
omega_error_min : 67/105
omega_error_max : 67/105
tperi : 79/105
tperi_error_min : 80/105
tperi_error_max : 80/105
tconj : 99/105
tconj_error_min : 99/105
tconj_error_max : 99/105
tzero_tr : 85/105
tzero_tr_error_min : 92/105
tzero_tr_error_max : 92/105
tzero_tr_sec : 105/105
tzero_tr_sec_error_min : 105/105
tzero_tr_sec_err

In [46]:
non_exo = non_exo[confirmed_planets.columns]

In [47]:
non_exo.head()

,mass,radius,radius_error_min,radius_error_max,orbital_period,orbital_period_error_min,orbital_period_error_max,semi_major_axis,ra,dec,...,star_mass,star_mass_error_min,star_mass_error_max,star_radius,star_radius_error_min,star_radius_error_max,star_age,star_teff,star_teff_error_min,star_teff_error_max
0,NaN,1.8,0.14,0.14,818000.0000,140000.0000,140000.0000,205.000,276.158333,30.429444,...,1.73,0.10,0.10,1.75,0.12,0.12,NaN,7800.0,NaN,NaN
1,148.0000,NaN,NaN,NaN,335.1000,2.5000,2.5000,0.995,39.241667,24.648056,...,1.22,0.08,0.08,1.23,0.07,0.07,0.91,6484.0,98.0,98.0
2,3.8800,NaN,NaN,NaN,479.1000,6.2000,6.2000,1.190,276.495833,65.563611,...,0.98,0.05,0.05,22.03,1.00,1.00,9.49,4200.0,70.0,70.0
3,0.0727,NaN,NaN,NaN,2.2246,0.0219,0.0219,0.025,154.900000,19.870000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6.4700,NaN,NaN,NaN,614.1000,1.1800,1.1800,1.460,68.979167,16.509167,...,1.13,0.11,0.11,45.10,0.10,0.10,6.60,4055.0,70.0,70.0


In [48]:
non_exo_non_nan_cols = []
for each_column in non_exo.columns:
    if non_exo[each_column].isna().sum()<50:
        non_exo[each_column] = non_exo[each_column].fillna(non_exo[each_column].median())
        non_exo_non_nan_cols.append(each_column)

In [49]:
non_nan_row_non_exo = non_exo.dropna()
null_non_exo = non_exo[non_exo.isnull().any(axis=1)]


(2, 27)
(103, 27)


In [50]:
nan_cols_non_exo = list(set(non_exo.columns.to_list()) - set(non_exo_non_nan_cols))


In [51]:
x_train = non_nan_row_non_exo[non_exo_non_nan_cols]
y_train = non_nan_row_non_exo[nan_cols_non_exo]

In [52]:
for each_column in non_exo.columns:        
    print(f"{each_column} : {non_exo[each_column].isna().sum()}/{len(non_exo[each_column])}")
print(non_exo.shape)

mass : 0/105
radius : 68/105
radius_error_min : 78/105
radius_error_max : 78/105
orbital_period : 0/105
orbital_period_error_min : 0/105
orbital_period_error_max : 0/105
semi_major_axis : 0/105
ra : 0/105
dec : 0/105
mag_j : 89/105
star_distance : 0/105
star_distance_error_min : 52/105
star_distance_error_max : 52/105
star_metallicity : 0/105
star_metallicity_error_min : 0/105
star_metallicity_error_max : 0/105
star_mass : 0/105
star_mass_error_min : 0/105
star_mass_error_max : 0/105
star_radius : 0/105
star_radius_error_min : 0/105
star_radius_error_max : 0/105
star_age : 0/105
star_teff : 0/105
star_teff_error_min : 0/105
star_teff_error_max : 0/105
(105, 27)


In [53]:
X_pred

,orbital_period,orbital_period_error_min,orbital_period_error_max,ra,dec,star_distance,star_mass,star_radius,star_teff,star_teff_error_min,star_teff_error_max
0,326.030000,0.3200,0.3200,185.179167,17.792778,110.6,2.7000,19.000,4742.0,100.00,100.00
1,730000.000000,365000.0000,365000.0000,245.604167,-24.087222,145.0,0.0162,0.960,2375.0,175.00,175.00
2,516.220000,3.2500,3.2500,229.275000,71.823889,119.5,1.8000,24.080,4340.0,70.00,70.00
3,185.840000,0.2300,0.2300,352.820833,39.236111,76.4,2.2000,11.000,4813.0,20.00,20.00
4,1767.560000,0.2200,0.2200,242.595833,43.821667,18.1,0.9000,0.708,5311.0,87.00,87.00
...,...,...,...,...,...,...,...,...,...,...,...
5633,0.037000,0.0001,0.0001,35.037500,21.698889,350.0,0.8300,0.010,14200.0,1000.00,1000.00
5634,0.024600,0.0001,0.0001,61.954167,-0.121389,810.0,0.7900,0.010,17400.0,1200.00,1200.00
5635,0.069789,0.0001,0.0001,245.736090,47.514220,365.5,0.4700,0.182,29000.0,88.55,88.55
5636,0.043000,0.0001,0.0001,249.433333,49.294722,207.0,0.9000,0.009,11200.0,300.00,300.00


In [54]:
model = LinearRegression()
model.fit(x_train, y_train)

X_pred = null_non_exo.drop(nan_cols_non_exo, axis =1)
print(X_pred.shape)
y_pred = model.predict(X_pred)
y_pred = pd.DataFrame(y_pred, columns = y_train.columns).applymap(lambda x: abs(x) if x < 0 else x)

(103, 21)


In [55]:
y_pred = y_pred.set_index(X_pred.index)

In [56]:
print(X_pred.isnull().values.any())

False


In [57]:
imputed_df = pd.concat([X_pred,y_pred], axis = 1)

final_non_exo_confirmed = pd.concat([non_nan_row_non_exo,imputed_df], axis = 0).sort_index()

(103, 27)
(2, 27)


In [58]:
final_non_exo_confirmed.shape

(105, 27)

In [59]:
imputed_df.head()

,mass,orbital_period,orbital_period_error_min,orbital_period_error_max,semi_major_axis,ra,dec,star_distance,star_metallicity,star_metallicity_error_min,...,star_age,star_teff,star_teff_error_min,star_teff_error_max,star_distance_error_min,radius_error_max,star_distance_error_max,radius_error_min,mag_j,radius
0,1.5400,818000.0000,140000.0000,140000.0000,205.000,276.158333,30.429444,389.000,-0.02,0.13,...,4.40,7800.0,72.12,72.12,31155.105491,13.893276,31155.105491,13.893276,219.159008,58.513614
1,148.0000,335.1000,2.5000,2.5000,0.995,39.241667,24.648056,39.400,0.12,0.08,...,0.91,6484.0,98.00,98.00,77.469153,0.051818,77.469153,0.051818,15.439879,0.429499
2,3.8800,479.1000,6.2000,6.2000,1.190,276.495833,65.563611,97.300,-0.46,0.05,...,9.49,4200.0,70.00,70.00,80.085473,0.024828,80.085473,0.024828,12.002829,0.026270
3,0.0727,2.2246,0.0219,0.0219,0.025,154.900000,19.870000,4.966,-0.02,0.13,...,4.40,5340.0,72.12,72.12,31.312467,0.001732,31.312467,0.001732,12.832451,0.140493
4,6.4700,614.1000,1.1800,1.1800,1.460,68.979167,16.509167,20.430,-0.27,0.05,...,6.60,4055.0,70.00,70.00,91.632805,0.029529,91.632805,0.029529,12.091362,0.012117


# Candidate planets dataset

In [60]:
candidate_planets.head()

,name,planet_status,mass,mass_error_min,mass_error_max,mass_sini,mass_sini_error_min,mass_sini_error_max,radius,radius_error_min,...,star_sp_type,star_age,star_age_error_min,star_age_error_max,star_teff,star_teff_error_min,star_teff_error_max,star_detected_disc,star_magnetic_field,star_alternate_names
0,11 Uma b,Candidate,3.72,0.82,0.82,NaN,NaN,NaN,NaN,NaN,...,K5III,NaN,NaN,NaN,4090.0,70.0,70.0,NaN,NaN,NaN
1,14 Her d,Candidate,0.28,NaN,NaN,0.28,NaN,NaN,NaN,NaN,...,K0 V,5.1,NaN,NaN,5311.0,87.0,87.0,NaN,NaN,NaN
2,19b1-02162 b,Candidate,NaN,NaN,NaN,NaN,NaN,NaN,1.97,NaN,...,G8,NaN,NaN,NaN,5500.0,NaN,NaN,NaN,NaN,NaN
3,19g-2-01326 b,Candidate,NaN,NaN,NaN,NaN,NaN,NaN,2.43,NaN,...,K1,NaN,NaN,NaN,5000.0,NaN,NaN,NaN,NaN,NaN
4,1SWASPJ115718.66+261906.1 b,Candidate,NaN,NaN,NaN,NaN,NaN,NaN,1.55,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
# Displaying Number of null vals for each columns

for each_column in candidate_planets.columns:        
    print(f"{each_column} : {candidate_planets[each_column].isna().sum()}/{len(candidate_planets[each_column])}")
print(candidate_planets.shape)



name : 0/2749
planet_status : 0/2749
mass : 2413/2749
mass_error_min : 2504/2749
mass_error_max : 2504/2749
mass_sini : 2628/2749
mass_sini_error_min : 2648/2749
mass_sini_error_max : 2648/2749
radius : 398/2749
radius_error_min : 436/2749
radius_error_max : 436/2749
orbital_period : 86/2749
orbital_period_error_min : 246/2749
orbital_period_error_max : 246/2749
semi_major_axis : 327/2749
semi_major_axis_error_min : 2617/2749
semi_major_axis_error_max : 2617/2749
eccentricity : 2547/2749
eccentricity_error_min : 2573/2749
eccentricity_error_max : 2573/2749
inclination : 410/2749
inclination_error_min : 2650/2749
inclination_error_max : 2650/2749
angular_distance : 2699/2749
discovered : 17/2749
updated : 0/2749
omega : 2602/2749
omega_error_min : 2613/2749
omega_error_max : 2613/2749
tperi : 2637/2749
tperi_error_min : 2645/2749
tperi_error_max : 2645/2749
tconj : 2748/2749
tconj_error_min : 2748/2749
tconj_error_max : 2748/2749
tzero_tr : 395/2749
tzero_tr_error_min : 487/2749
tzero_t

In [62]:
candidate_planets = candidate_planets[confirmed_planets.columns]

candidate_planets.head()



,mass,radius,radius_error_min,radius_error_max,orbital_period,orbital_period_error_min,orbital_period_error_max,semi_major_axis,ra,dec,...,star_mass,star_mass_error_min,star_mass_error_max,star_radius,star_radius_error_min,star_radius_error_max,star_age,star_teff,star_teff_error_min,star_teff_error_max
0,3.72,NaN,NaN,NaN,651.900000,7.100000,7.100000,1.56,137.095833,66.873333,...,1.81,0.195,0.195,36.300,4.200,4.200,NaN,4090.0,70.0,70.0
1,0.28,NaN,NaN,NaN,3765.000000,77.000000,77.000000,NaN,242.595833,43.821667,...,0.90,NaN,NaN,0.708,0.085,0.085,5.1,5311.0,87.0,87.0
2,NaN,1.97,NaN,NaN,0.598627,NaN,NaN,NaN,293.011200,36.484800,...,0.95,0.070,0.070,1.120,0.070,0.070,NaN,5500.0,NaN,NaN
3,NaN,2.43,NaN,NaN,0.430000,NaN,NaN,NaN,294.273400,36.258700,...,0.84,NaN,NaN,0.980,0.100,0.100,NaN,5000.0,NaN,NaN
4,NaN,1.55,NaN,NaN,1.226804,0.000128,0.000128,NaN,179.329167,26.318333,...,NaN,NaN,NaN,1.320,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
# filling values in columns with median 

candidate_planets_non_nan_cols = []
for each_column in candidate_planets.columns:
    if candidate_planets[each_column].isna().sum()<1500:
        candidate_planets[each_column] = candidate_planets[each_column].fillna(candidate_planets[each_column].median())
        candidate_planets_non_nan_cols.append(each_column)




In [64]:
# Separating dataframes based on if any row contains any null value or not

non_nan_candidate_planets = candidate_planets.dropna()
null_non_exo = candidate_planets[candidate_planets.isnull().any(axis=1)]


# Separating nan columns from non nan columns
nan_cols_candidate_planets = list(set(candidate_planets.columns.to_list()) - set(candidate_planets_non_nan_cols))


In [65]:
x_train = non_nan_candidate_planets[candidate_planets_non_nan_cols]
y_train = non_nan_candidate_planets[nan_cols_candidate_planets]

In [66]:
model = LinearRegression()
model.fit(x_train, y_train)

LinearRegression()

In [67]:
X_pred = null_non_exo.drop(nan_cols_candidate_planets, axis =1)
X_pred = X_pred.reset_index(drop=True)
y_pred = model.predict(X_pred)

In [68]:
candidate_planets_2 = pd.read_csv("Candidate_planets.csv")

In [70]:

y_pred = pd.DataFrame(y_pred, columns = y_train.columns).applymap(lambda x: abs(x) if x < 0 else x)

In [71]:
y_pred

,star_distance_error_min,mass,star_distance_error_max,star_distance,star_age
0,20.118557,8.981474,20.118557,157.585186,3.057870
1,6.750922,27.616321,6.750922,207.718887,3.307331
2,37.939414,27.698910,37.939414,295.588227,2.254651
3,43.574047,34.157589,43.574047,316.144933,1.479992
4,32.787775,25.671858,32.787775,274.795295,1.975168
...,...,...,...,...,...
2658,6.842856,17.377606,6.842856,92.880594,3.396838
2659,25.148154,6.557112,25.148154,267.897665,5.511452
2660,7.494158,100.948485,7.494158,35.035507,4.260526
2661,6.010220,0.077185,6.010220,45.906524,3.346277


In [72]:
imputed_df = pd.concat([X_pred,y_pred], axis = 1)
final_candidate = pd.concat([non_nan_candidate_planets,imputed_df], axis = 0)

final_candidate.shape

(2749, 27)

# Prepping final train and test dataset

In [73]:
def add_name_status_back(df,main_df):
    planet_status, name = [],[]

    for index,rows in main_df.iterrows():
        if index in df.index:
            planet_status.append(rows['planet_status'])
            name.append(rows['name'])
    print(main_df.shape)
    print(df.shape)
    print(len(name))
    df['name'] = name
    df['planet_status'] = planet_status
    return df

In [74]:
non_exo = pd.read_csv("Confirmed_non_exoplanets.csv")
exo_confirmed = pd.read_csv("Confirmed_planets.csv")

In [75]:
finalest_non_exo = add_name_status_back(final_non_exo_confirmed, non_exo)


(105, 98)
(105, 27)
105


In [76]:
final_confirmed_exo = add_name_status_back(final_confirmed_exo, exo_confirmed)

(5638, 98)
(5638, 27)
5638


In [77]:
final_confirmed_exo.head()

,mass,radius,radius_error_min,radius_error_max,orbital_period,orbital_period_error_min,orbital_period_error_max,semi_major_axis,ra,dec,...,star_mass_error_max,star_radius,star_radius_error_min,star_radius_error_max,star_age,star_teff,star_teff_error_min,star_teff_error_max,name,planet_status
0,3.308195,1.943048,0.171463,0.171463,326.03,0.32,0.32,0.720440,185.179167,17.792778,...,0.472119,19.000,0.831434,0.831434,11.449135,4742.0,100.0,100.0,11 Com b,Confirmed
1,180.143765,19.624229,2.713336,2.713336,730000.00,365000.00,365000.00,258.295623,245.604167,-24.087222,...,0.816488,0.960,1.971924,1.971924,46.808941,2375.0,175.0,175.0,11 Oph b,Confirmed
2,4.064879,1.216779,0.204471,0.204471,516.22,3.25,3.25,1.162126,229.275000,71.823889,...,0.514292,24.080,1.201410,1.201410,27.059812,4340.0,70.0,70.0,11 UMi b,Confirmed
3,4.500641,1.118902,0.088233,0.088233,185.84,0.23,0.23,0.455490,352.820833,39.236111,...,0.301509,11.000,0.432204,0.432204,5.157533,4813.0,20.0,20.0,14 And b,Confirmed
4,7.733502,0.883573,0.108311,0.108311,1767.56,0.22,0.22,3.501505,242.595833,43.821667,...,0.037001,0.708,0.038252,0.038252,3.166228,5311.0,87.0,87.0,14 Her b,Confirmed


In [92]:
# train = final_non_exo + final_exo
planet_status = []
train_data = pd.concat([finalest_non_exo, final_confirmed_exo], axis = 0)

In [93]:
# Processing the target variable

for each_val in train_data['planet_status']:
    if each_val == 'Confirmed':
        planet_status.append(1)
    else:
        planet_status.append(0)

train_data['planet_status'] = planet_status
train_planet_names = train_data['name']
train_data = train_data.drop(['name'], axis= 1)
train_data = train_data.sample(frac=1)
y_train = train_data['planet_status']
x_train = train_data.drop(['planet_status'], axis= 1)

train_data.head()

,mass,radius,radius_error_min,radius_error_max,orbital_period,orbital_period_error_min,orbital_period_error_max,semi_major_axis,ra,dec,...,star_mass_error_min,star_mass_error_max,star_radius,star_radius_error_min,star_radius_error_max,star_age,star_teff,star_teff_error_min,star_teff_error_max,planet_status
1741,0.932328,0.751015,0.040112,0.040112,1.921036,0.000053,0.000053,0.049261,13.194414,9.692924,...,0.034297,0.034297,0.82,0.001710,0.001710,4.805023,5060.0,70.00,70.00,1
100,1.540000,0.008733,0.030238,0.030238,121.710000,0.305000,0.305000,0.431500,132.487500,11.692500,...,0.050000,0.050000,21.80,0.700000,0.700000,4.400000,4284.0,9.00,9.00,0
2892,5.458350,1.315946,0.071027,0.071027,20.684342,0.000097,0.000097,0.147552,289.830114,50.586246,...,0.050600,0.050600,1.15,0.104671,0.104671,4.575505,6250.0,200.00,200.00,1
2643,1.162564,0.343070,0.017339,0.017339,3.951169,0.000016,0.000016,0.103515,285.728495,42.654530,...,0.052729,0.052729,0.99,0.048522,0.048522,5.328160,5564.0,63.29,63.29,1
3439,5.783454,0.986597,0.059062,0.059062,3.282807,0.000020,0.000020,0.112175,294.137930,42.370502,...,0.043151,0.043151,0.91,0.122347,0.122347,6.755001,5977.0,200.00,200.00,1


In [99]:
# Performing oversampling to balance the data


print('Original dataset shape ', train_data.shape)

# Apply SMOTE
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = smote.fit_resample(x_train, y_train)

print('Resampled dataset shape ', pd.concat([X_resampled, y_resampled], axis = 1).shape)


Original dataset shape  (5743, 28)
Resampled dataset shape  (11276, 28)


In [108]:
new_dataset = pd.concat([X_resampled, y_resampled], axis = 1)
new_dataset.head()

,mass,radius,radius_error_min,radius_error_max,orbital_period,orbital_period_error_min,orbital_period_error_max,semi_major_axis,ra,dec,...,star_mass_error_min,star_mass_error_max,star_radius,star_radius_error_min,star_radius_error_max,star_age,star_teff,star_teff_error_min,star_teff_error_max,planet_status
0,0.932328,0.751015,0.040112,0.040112,1.921036,0.000053,0.000053,0.049261,13.194414,9.692924,...,0.034297,0.034297,0.82,0.001710,0.001710,4.805023,5060.0,70.00,70.00,1
1,1.540000,0.008733,0.030238,0.030238,121.710000,0.305000,0.305000,0.431500,132.487500,11.692500,...,0.050000,0.050000,21.80,0.700000,0.700000,4.400000,4284.0,9.00,9.00,0
2,5.458350,1.315946,0.071027,0.071027,20.684342,0.000097,0.000097,0.147552,289.830114,50.586246,...,0.050600,0.050600,1.15,0.104671,0.104671,4.575505,6250.0,200.00,200.00,1
3,1.162564,0.343070,0.017339,0.017339,3.951169,0.000016,0.000016,0.103515,285.728495,42.654530,...,0.052729,0.052729,0.99,0.048522,0.048522,5.328160,5564.0,63.29,63.29,1
4,5.783454,0.986597,0.059062,0.059062,3.282807,0.000020,0.000020,0.112175,294.137930,42.370502,...,0.043151,0.043151,0.91,0.122347,0.122347,6.755001,5977.0,200.00,200.00,1


In [109]:
new_dataset = pd.concat([X_resampled, y_resampled], axis = 1)
X = new_dataset.drop(['planet_status'], axis =1)
y = new_dataset['planet_status']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [117]:
new_dataset.shape

(11276, 28)

# Performing Cross Validation using various Classifications models and compare Scores

In [150]:
models = [
    ('decision_tree', DecisionTreeClassifier()),
    ('svm', SVC()),
    ('random_forest', RandomForestClassifier()),
    ('xgboost', XGBClassifier()),
    ('logistic_regression', LogisticRegression())
]

# Voting Classifier
voting_classifier = VotingClassifier(estimators=models, voting='hard')

param_grids = [
    # Decision Tree
    {
        'max_depth': [3, 5, 7]
    },
    # SVM
    {
        'C': [0.1, 1, 10],
        'kernel': ['linear', 'rbf']
    },
    # Random Forest
    {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 5, 7]
    },
    # XGBoost
    {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.1, 0.2]
    },
    # Logistic Regression
    {
        'C': [0.1, 1, 10]
    }
]

# Define GridSearchCV for each model
grid_searches = []
for model, param_grid in zip(models, param_grids):
    grid_search = GridSearchCV(model[1], param_grid, cv=5, scoring='f1')
    grid_search.fit(X_train, y_train)
    grid_searches.append((model[0], grid_search))
    
for model_name, grid_search in grid_searches:
    print("Model:", model_name)
    print("Best parameters:", grid_search.best_params_)
    print("Best F1 score:", grid_search.best_score_)

# Evaluate Voting Classifier
voting_score = cross_val_score(voting_classifier, X_test, y_test, cv=5, scoring='f1')
print("Voting Classifier F1 Score:", voting_score.mean())

/Users/ganeshmadduri/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/ganeshmadduri/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.

Model: decision_tree
Best parameters: {'max_depth': 5}
Best F1 score: 0.9655708180708181

Model: svm
Best parameters: {'C': 1, 'kernel': 'linear'}
Best F1 score: 0.9749909812409813

Model: random_forest
Best parameters: {'max_depth': 5, 'n_estimators': 200}
Best F1 score: 0.9745967741935484

Model: xgboost
Best parameters: {'learning_rate': 0.1, 'n_estimators': 100}
Best F1 score: 0.9807393312516263

Model: logistic_regression
Best parameters: {'C': 10}
Best F1 score: 0.8815744646333549

Voting Classifier F1 Score: 0.9154147812971342


/Users/ganeshmadduri/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/ganeshmadduri/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.

In [165]:
# Define the parameter grid for hyperparameter tuning
param_grid = {
    'hidden_layer_sizes': [(100,), (50, 50), (50, 100, 50)],
    'activation': ['relu', 'tanh', 'logistic'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.0001, 0.0002,0.0003,0.0004,0.0005, 0.001, 0.01],
    'learning_rate': ['constant', 'adaptive'],
}

# Create an MLPClassifier
mlp = MLPClassifier(max_iter=1000)

# Perform GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(mlp, param_grid, cv=5, scoring='f1', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and the corresponding F1 score
print("Best hyperparameters:", grid_search.best_params_)
print("Best F1 score:", grid_search.best_score_)

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
test_score = best_model.score(X_test, y_test)
print("Test set accuracy of the best model:", test_score)

/Users/ganeshmadduri/anaconda3/lib/python3.11/site-packages/sklearn/utils/extmath.py:193: RuntimeWarning: overflow encountered in matmul
  ret = a @ b


Best hyperparameters: {'activation': 'logistic', 'alpha': 0.0004, 'hidden_layer_sizes': (100,), 'learning_rate': 'adaptive', 'solver': 'adam'}
Best F1 score: 0.867072266760586
Test set accuracy of the best model: 0.8987341772151899


In [157]:
models = [('lr', LogisticRegression(C=10)), 
          ('dt', DecisionTreeClassifier(max_depth=5)), 
          ('svc', SVC(C=1, kernel='linear')), 
          ('xgb', XGBClassifier(learning_rate=0.1, n_estimators=100)), 
          ('rf', RandomForestClassifier(max_depth=5, n_estimators=200))]

In [161]:
voting_classifier = VotingClassifier(estimators=models, voting='hard')
voting_classifier = voting_classifier.fit(X_train, y_train)
y_pred = voting_classifier.predict(X_test)
print(f1_score(y_test, y_pred))

/Users/ganeshmadduri/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


1.0


In [162]:
models = [LogisticRegression(C=10), 
          DecisionTreeClassifier(max_depth=5), 
          SVC(C=1, kernel='linear'), 
          XGBClassifier(learning_rate=0.1, n_estimators=100), 
          RandomForestClassifier(max_depth=5, n_estimators=200)]

In [163]:
for each_model in models:   
    each_model = each_model.fit( X_train, y_train)
    y_pred = each_model.predict(X_test)
    scores = f1_score(y_test, y_pred)
    print(f"{scores} for {each_model}")

/Users/ganeshmadduri/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.891566265060241 for LogisticRegression(C=10)
0.9743589743589743 for DecisionTreeClassifier(max_depth=5)
1.0 for SVC(C=1, kernel='linear')
0.9870129870129869 for XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)
0.9743589743589743 for RandomForestClassifier(max_depth=5, n_estimators=200)


In [ ]:
# Metrics to use :

# - AUC-ROC
# - F1

# Do gridsearchCV for each chosen model on its hyperparameters
# Then do cv score again
# Add ANN
# Then choose the best cv score model 
# Make prediction on Candidate planets


# The following is the result of hyperparameter tuning using GridsearchCross Validation

## The results provide an overview of how each model performed on the dataset


**Model**: decision_tree
**Best parameters**: {'max_depth': 5}
**Best F1 score**: 0.9655708180708181

**Model**: svm
**Best parameters**: {'C': 1, 'kernel': 'linear'}
**Best F1 score**: 0.9749909812409813

**Model**: random_forest
**Best parameters**: {'max_depth': 5, 'n_estimators': 200}
**Best F1 score**: 0.9745967741935484

**Model**: xgboost
**Best parameters**: {'learning_rate': 0.1, 'n_estimators': 100}
**Best F1 score**: 0.9807393312516263

**Model**: logistic_regression
**Best parameters**: {'C': 10}
**Best F1 score**: 0.8815744646333549

**Model**: ANN
**Best hyperparameters**: {'activation': 'logistic', 'alpha': 0.0004, 'hidden_layer_sizes': (100,), 'learning_rate': 'adaptive', 'solver': 'adam'}
**Best F1 score**: 0.867072266760586

### I will be using the top 2 models to make predictions and compare their results



In [166]:
X_train.head()

,mass,radius,radius_error_min,radius_error_max,orbital_period,orbital_period_error_min,orbital_period_error_max,semi_major_axis,ra,dec,...,star_mass,star_mass_error_min,star_mass_error_max,star_radius,star_radius_error_min,star_radius_error_max,star_age,star_teff,star_teff_error_min,star_teff_error_max
3354,1.506363,0.615451,0.023282,0.023282,26.730000,0.020000,0.020000,0.138005,359.975000,-22.428056,...,1.330000,0.100508,0.100508,1.700000,0.086329,0.086329,3.869744,6037.000000,44.000000,44.000000
5895,0.073005,0.131447,0.059778,0.059778,29.358049,0.069883,0.069883,0.319440,230.013421,-27.386909,...,0.311375,0.023170,0.023170,0.330117,0.050790,0.050790,4.847921,3435.378946,193.276318,193.276318
10677,1.540000,0.174534,0.006703,0.006703,13.772696,0.030696,0.030696,0.316746,294.608801,44.374244,...,0.876658,0.067571,0.067571,1.055949,0.156066,0.156066,4.400000,5544.885653,134.230619,134.230619
8754,1.540000,0.303338,0.034124,0.034124,30.557218,0.015859,0.015859,0.856954,217.468307,43.747123,...,1.024515,0.066891,0.066891,0.829743,0.070000,0.070000,4.517149,5333.108909,71.513584,71.513584
3436,3.789162,0.890315,0.053511,0.053511,26.964432,0.000157,0.000157,0.151006,294.366727,42.252678,...,0.900000,0.039986,0.039986,0.870000,0.074303,0.074303,5.210830,5508.000000,129.720000,129.720000


In [168]:
final_candidate.head()

,mass,radius,radius_error_min,radius_error_max,orbital_period,orbital_period_error_min,orbital_period_error_max,semi_major_axis,ra,dec,...,star_mass,star_mass_error_min,star_mass_error_max,star_radius,star_radius_error_min,star_radius_error_max,star_age,star_teff,star_teff_error_min,star_teff_error_max
14,6.000,0.183,0.02,0.02,15.068059,0.000116,0.000116,6.00000,94.800000,-58.054167,...,0.972,0.106,0.106,0.967,0.091,0.091,0.030,5285.0,9.0,9.0
18,6.600,0.183,0.02,0.02,128.200000,0.100000,0.100000,0.97000,51.079167,49.861111,...,7.300,0.300,0.300,55.000,0.091,0.091,0.041,6350.0,100.0,100.0
23,0.111,0.183,0.02,0.02,33.400000,0.100000,0.100000,0.11485,311.291667,-31.340833,...,0.500,0.030,0.030,0.750,0.030,0.030,0.022,3700.0,100.0,100.0
26,30.000,0.183,0.02,0.02,15.068059,0.000116,0.000116,0.11485,350.604167,-2.228056,...,1.300,0.106,0.106,0.967,0.091,0.091,0.007,5796.0,165.0,165.0
53,30.000,0.183,0.02,0.02,15.068059,0.000116,0.000116,9.80000,284.962272,-37.108753,...,0.972,0.106,0.106,0.967,0.091,0.091,7.500,5796.0,165.0,165.0


In [171]:
X_test.shape

(79, 27)

In [172]:
final_candidate_dataset = pd.concat([final_candidate, X_test], axis = 0)


In [173]:
model_1 = RandomForestClassifier(max_depth=5, n_estimators=200)
model_1 = model_1.fit(X_train, y_train)
model_1_y_pred = model_1.predict(final_candidate_dataset)


In [183]:
known_y_pred_1 = model_1_y_pred[-79:]
y_test_1 = list(y_test)
acc = 0
for i in range(len(known_y_pred_1)):
    if known_y_pred_1[i] == y_test_1[i]:
        acc+=1
    
print(f"Accuracy % is {(acc/len(y_test))*100}")

79
79
Accuracy % is 98.73417721518987


In [185]:
model_2 = XGBClassifier(learning_rate=0.1, n_estimators=100)
model_2 = model_2.fit(X_train, y_train)
model_2_y_pred = model_1.predict(final_candidate_dataset)

In [186]:
known_y_pred_2 = model_2_y_pred[-79:]

acc = 0
for i in range(len(known_y_pred_2)):
    if known_y_pred_2[i] == y_test_1[i]:
        acc+=1
    
print(f"Accuracy % is {(acc/len(y_test))*100}")

Accuracy % is 98.73417721518987


In [187]:
matches = 0
for i in range(len(known_y_pred_2)):
    if known_y_pred_2[i] == known_y_pred_1[i]:
        matches+=1
        
print(f"Matched % is {(matches/len(y_test))*100}")

Matched % is 100.0


In [190]:
matches = 0
for i in range(len(model_1_y_pred)):
    if model_1_y_pred[i] == model_2_y_pred[i]:
        matches+=1
        
print(f"Matched % is {(matches/len(model_1_y_pred))*100}")

Matched % is 100.0


## It is clear that both models made the same predictions with the same accuracy

## I will now create a final dataframe filled with exoplanet prediction intact and the display the planet names that are predicted to be exoplanets

In [191]:
final_candidate_dataset['Planet Status'] = model_1_y_pred

In [199]:
## The test set was added to the candidate dataset to check how well the model performed 
## Since we don't need it now, we are removing it

final_final_candidate_dataset = final_candidate_dataset[:-79]

In [204]:
test_candid = candidate_planets_2.iloc[final_final_candidate_dataset.index]

,name,planet_status,mass,mass_error_min,mass_error_max,mass_sini,mass_sini_error_min,mass_sini_error_max,radius,radius_error_min,...,star_sp_type,star_age,star_age_error_min,star_age_error_max,star_teff,star_teff_error_min,star_teff_error_max,star_detected_disc,star_magnetic_field,star_alternate_names
14,AB Pic c,Candidate,6.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,K2 V,0.030,NaN,NaN,5285.0,9.0,9.0,NaN,NaN,NaN
18,alf Per b,Candidate,6.600,0.200,0.200,NaN,NaN,NaN,NaN,NaN,...,F5Ib,0.041,NaN,NaN,6350.0,100.0,100.0,NaN,NaN,NaN
23,AU Mic e,Candidate,0.111,0.017,0.017,0.111,0.017,0.017,NaN,NaN,...,M1V,0.022,0.003,0.003,3700.0,100.0,100.0,NaN,NaN,NaN
26,BP Psc b,Candidate,30.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,O9III,0.007,0.007,0.007,NaN,NaN,NaN,NaN,NaN,NaN
53,DENIS J185950.9-370632 b,Candidate,30.000,10.000,10.000,NaN,NaN,NaN,NaN,NaN,...,M8V,7.500,2.500,2.500,NaN,NaN,NaN,NaN,NaN,"2MASSW J1858509-370631, 2MASS J18595094-3706313"


In [205]:
absolute_predicted_data = final_final_candidate_dataset
absolute_predicted_data['Planet_name'] = test_candid['name']
absolute_predicted_data.head()

/var/folders/02/f2x5vw_s3_bbmffg9fz72r9r0000gn/T/ipykernel_31860/1858304525.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  absolute_predicted_data['Planet_name'] = test_candid['name']


,mass,radius,radius_error_min,radius_error_max,orbital_period,orbital_period_error_min,orbital_period_error_max,semi_major_axis,ra,dec,...,star_mass_error_max,star_radius,star_radius_error_min,star_radius_error_max,star_age,star_teff,star_teff_error_min,star_teff_error_max,Planet Status,Planet_name
14,6.000,0.183,0.02,0.02,15.068059,0.000116,0.000116,6.00000,94.800000,-58.054167,...,0.106,0.967,0.091,0.091,0.030,5285.0,9.0,9.0,1,AB Pic c
18,6.600,0.183,0.02,0.02,128.200000,0.100000,0.100000,0.97000,51.079167,49.861111,...,0.300,55.000,0.091,0.091,0.041,6350.0,100.0,100.0,0,alf Per b
23,0.111,0.183,0.02,0.02,33.400000,0.100000,0.100000,0.11485,311.291667,-31.340833,...,0.030,0.750,0.030,0.030,0.022,3700.0,100.0,100.0,0,AU Mic e
26,30.000,0.183,0.02,0.02,15.068059,0.000116,0.000116,0.11485,350.604167,-2.228056,...,0.106,0.967,0.091,0.091,0.007,5796.0,165.0,165.0,0,BP Psc b
53,30.000,0.183,0.02,0.02,15.068059,0.000116,0.000116,9.80000,284.962272,-37.108753,...,0.106,0.967,0.091,0.091,7.500,5796.0,165.0,165.0,0,DENIS J185950.9-370632 b


In [206]:
absolute_predicted_data.columns

Index(['mass', 'radius', 'radius_error_min', 'radius_error_max',
       'orbital_period', 'orbital_period_error_min',
       'orbital_period_error_max', 'semi_major_axis', 'ra', 'dec', 'mag_j',
       'star_distance', 'star_distance_error_min', 'star_distance_error_max',
       'star_metallicity', 'star_metallicity_error_min',
       'star_metallicity_error_max', 'star_mass', 'star_mass_error_min',
       'star_mass_error_max', 'star_radius', 'star_radius_error_min',
       'star_radius_error_max', 'star_age', 'star_teff', 'star_teff_error_min',
       'star_teff_error_max', 'Planet Status', 'Planet_name'],
      dtype='object')

In [207]:
# Rearranging the columns to have planet name and planet status in the front

absolute_predicted_data = absolute_predicted_data[['Planet_name','Planet Status','mass', 'radius', 'radius_error_min', 'radius_error_max',
       'orbital_period', 'orbital_period_error_min',
       'orbital_period_error_max', 'semi_major_axis', 'ra', 'dec', 'mag_j',
       'star_distance', 'star_distance_error_min', 'star_distance_error_max',
       'star_metallicity', 'star_metallicity_error_min',
       'star_metallicity_error_max', 'star_mass', 'star_mass_error_min',
       'star_mass_error_max', 'star_radius', 'star_radius_error_min',
       'star_radius_error_max', 'star_age', 'star_teff', 'star_teff_error_min',
       'star_teff_error_max' ]]

In [208]:
absolute_predicted_data.head()

,Planet_name,Planet Status,mass,radius,radius_error_min,radius_error_max,orbital_period,orbital_period_error_min,orbital_period_error_max,semi_major_axis,...,star_mass,star_mass_error_min,star_mass_error_max,star_radius,star_radius_error_min,star_radius_error_max,star_age,star_teff,star_teff_error_min,star_teff_error_max
14,AB Pic c,1,6.000,0.183,0.02,0.02,15.068059,0.000116,0.000116,6.00000,...,0.972,0.106,0.106,0.967,0.091,0.091,0.030,5285.0,9.0,9.0
18,alf Per b,0,6.600,0.183,0.02,0.02,128.200000,0.100000,0.100000,0.97000,...,7.300,0.300,0.300,55.000,0.091,0.091,0.041,6350.0,100.0,100.0
23,AU Mic e,0,0.111,0.183,0.02,0.02,33.400000,0.100000,0.100000,0.11485,...,0.500,0.030,0.030,0.750,0.030,0.030,0.022,3700.0,100.0,100.0
26,BP Psc b,0,30.000,0.183,0.02,0.02,15.068059,0.000116,0.000116,0.11485,...,1.300,0.106,0.106,0.967,0.091,0.091,0.007,5796.0,165.0,165.0
53,DENIS J185950.9-370632 b,0,30.000,0.183,0.02,0.02,15.068059,0.000116,0.000116,9.80000,...,0.972,0.106,0.106,0.967,0.091,0.091,7.500,5796.0,165.0,165.0


In [209]:
print(f"The following are the planets predicted to be exoplanets: \n")
for index,row in absolute_predicted_data.iterrows():
    if row['Planet Status'] == 1:
        print(row['Planet_name'])

The following are the planets predicted to be exoplanets: 

AB Pic c
HAT-P-26 c
HAT-P-37 c
HAT-P-7 c
HD 10180 b
HD 163607 d
HD 180902 c
HIP 10278 c
HIP 21934 b
Kepler-538 c
Kepler-93 d
KOI-1089.02
Qatar-2 c
tau Cet i
TOI-1136 h
TOI-1238 d
TOI-2196 c
TOI-4481 d
TrES-5 c
TW Hya c
TW Hya d
TW Hya e
TW Hya f
WASP-10 c
WASP-12 c
1SWASPJ115718.66+261906.1 b
1SWASPJ130322.00+350525.4 b
1SWASPJ160242.43+290850.1 b
1SWASPJ161732.90+242119.0 b
2M 1759-2739 b
alf Per b
CS Cha b
CS Cha c
CSTAR0012 b
CSTAR0751 b
DV Uma AB b
EPIC 211939692.03
EPIC 246251988 b
eps Psc b
ET Cha b
Gaia-ASOI-060 b
GD 552 b
GJ 911 b
GRS 1747-312 b
HD 107146 b
HD 131664 b
HD 180902 c
HD 184860 b
HD 18742 c
HD 18884 b
HD 191760 b
HD 191760 c
HD 196645 b
HD 197083 b
HD 205577 b
HD 206893 d
HD 207077 b
HD 20794 g
HD 208742 b
HD 3016 b
HD 30177 d
HD 3221 b
HD 38677 f
HD 3964 b
HD 4917 d
HD 52030 b
HD 57411 b
HD 6037 b
HD 74156 d
HD 98800B b
HIP 108095 b
HIP 13366 b
HIP 14342 b
HIP 14342 c
HIP 21934 b
HIP 30314 b
HIP 41378 g
H

In [212]:
len(absolute_predicted_data[absolute_predicted_data['Planet Status']==1])

909

In [ ]:
# Therefore out of the 2749 candidate planets 909 planets are predicted to be exoplanets with a 98% accuracy